In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from datetime import date, timedelta
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn_extra.cluster import KMedoids
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch_geometric.nn as geom_nn
from torch.nn import Linear
from torch_geometric.nn import dense_mincut_pool, ChebConv
from scipy.linalg import sqrtm, inv

# Constructing Datasets

In [2]:
coords = pd.read_csv('Data/Power Network Topology-full network (188 nodes)/Electricity Sytem Nodes-location.csv')
coords_agg = coords.drop_duplicates(subset=['lat', 'lon']) # remove duplicate coordinates

# read in electricity data
elec = pd.read_csv('Data/Power Network Topology-full network (188 nodes)/bus_load_RM_2050.csv', index_col=[0])
elec = pd.DataFrame(StandardScaler().fit_transform(elec)) # standardizing

# create hourly datetime index
times = pd.date_range(start='2050-01-01', end = '2051-01-01', freq='H')[:-1]
elec.index = times

# NG data
ng = pd.read_csv('Data/ng_daily_load2050_RM.csv', index_col=[0])
ng.index = pd.date_range(start='2050-01-01', end = '2050-12-31', freq='d')
ng = pd.DataFrame(StandardScaler().fit_transform(ng)) # standardizing

# solar data
solar = pd.read_csv('Data/solar-CF-188-nodes.csv', index_col=[0])
solar.index = pd.date_range(start='2016-01-01', end = '2017-01-01', freq='H')[:-1]
solar = solar.loc[solar.index.date != pd.to_datetime('2016-02-29')]
solar = pd.DataFrame(StandardScaler().fit_transform(solar)) # standardizing
solar.index = times

# wind data
wind = pd.read_csv('Data/wind-CF-188-nodes.csv', index_col=[0])
wind.index = pd.date_range(start='2016-01-01', end = '2017-01-01', freq='H')[:-1]
wind = wind.loc[wind.index.date != pd.to_datetime('2016-02-29')]
wind = pd.DataFrame(StandardScaler().fit_transform(wind)) # standardizing
wind.index = times

# remove columns for nodes at same coordinates
elec = elec[coords_agg.bus_num]
solar = solar[coords_agg.bus_num]
wind = wind[coords_agg.bus_num]

C:\Users\Rahman Khorramfar\AppData\Local\Temp\ipykernel_10712\1922429778.py:20: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  solar = solar.loc[solar.index.date != pd.to_datetime('2016-02-29')]
C:\Users\Rahman Khorramfar\AppData\Local\Temp\ipykernel_10712\1922429778.py:27: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  wind = wind.loc[wind.index.date != pd.to_datetime('2016-02-29')]


In [3]:
X_E = torch.zeros(365,len(elec.columns),24)
X_S = torch.zeros(365,len(solar.columns),24)
X_W = torch.zeros(365,len(wind.columns),24)
X_NG = torch.tensor(ng.to_numpy(), dtype=torch.float)

for date in range(365):
    for hour in range(24):
        for node in range(len(elec.columns)):
            X_E[date,node,hour] = elec.to_numpy()[date*24+hour, node]
            X_S[date,node,hour] = solar.to_numpy()[date*24+hour, node]
            X_W[date,node,hour] = wind.to_numpy()[date*24+hour, node]

# Spatial Aggregation Autoencoder

In [4]:
src = 'Data/Power Network Topology-full network (188 nodes)/'
edges = pd.read_csv(src + 'Branches Between Electricity Nodes.csv')
edges = edges.loc[edges['if this branch is existing'] == 1] # do not consider candidate branches

# create power/NG network graph
G_net = nx.Graph()
for node in coords_agg.bus_num:
    G_net.add_node(node, pos = coords_agg.loc[coords_agg.bus_num==node][['lat','lon']].values.flatten())

for x in G_net.nodes:
    for y in G_net.nodes:
        x_lat, x_lon = G_net.nodes[x]['pos']
        y_lat, y_lon = G_net.nodes[y]['pos']
        df = edges.loc[(edges['from bus'].isin(coords.loc[(coords.lat==x_lat) & 
                                                          (coords.lon==x_lon)].bus_num)) &
                       (edges['to bus'].isin(coords.loc[(coords.lat==y_lat) & 
                                                        (coords.lon==y_lon)].bus_num))]
        if x == y:
            G_net.add_edge(x, y, distance=0) # add self edge
        elif len(df) > 0:
            G_net.add_edge(x, y, distance=df['Distance (mile)'].mean()) # add edge with distance
            
edge_ix = torch.tensor(np.array(nx.convert_node_labels_to_integers(G_net.copy()).edges),dtype=torch.long).t().contiguous()

In [5]:
# initialize geospatial graph
G_geo = nx.Graph()
for node in coords_agg.bus_num:
    G_geo.add_node(node, pos = coords_agg.loc[coords_agg.bus_num==node][['lat','lon']].values.flatten())

# create list of distances
distances = list()
for x in G_geo.nodes:
    for y in G_geo.nodes:
        if x != y:
            (x1, y1) = G_geo.nodes[x]['pos']
            (x2, y2) = G_geo.nodes[y]['pos']
            dist = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            distances.append(dist)

# normalize distances
i = 0
sigma = np.std(distances)
for x in G_geo.nodes:
    for y in G_geo.nodes:
        if x != y:
            G_geo.add_edge(x, y, distance=np.exp((-distances[i]**2) / (sigma**2)))
            i += 1

In [6]:
class Spatial_AE(torch.nn.Module):

    def __init__(self, clusters):
        super(Spatial_AE, self).__init__()

        self.pool_conv1 = ChebConv(24 + len(G_net.nodes), clusters*3, K=1)
        self.pool_conv2 = ChebConv(clusters*3, clusters*2, K=1)
        self.pool_conv3 = Linear(clusters*2, clusters)

        # Electricity Convolution Layers
        self.conv1 = Linear(24, 18)
        self.conv2 = Linear(18, 12)

        encoder_params=0
        for p in list(self.parameters()):
            n=1
            for s in list(p.size()):
                n = n*s
            encoder_params += n
        print("Encoder Parameters: " + str(encoder_params))

        # Electricity Block Decoder Parameters
        self.deconv3 = Linear(12, 18)
        self.deconv4 = Linear(18, 24)

        decoder_params = 0
        for p in list(self.parameters()):
            n=1
            for s in list(p.size()):
                n = n*s
            decoder_params += n
        print("Decoder Parameters: " + str(decoder_params - encoder_params))

        print("Total Parameters: " + str(decoder_params))

    def forward(self, X, edge_ix, A_net, A_geo):

        X_mean = X.mean(dim=1, keepdim=True)
        X_norm = X - X_mean
        X_norm = torch.cat((X_norm, 
                            torch.eye(X_norm.shape[1]).repeat([X_norm.shape[0],1,1])),
                            dim=-1)

        X = self.conv1(X).tanh()
        X = self.conv2(X)

        S = self.pool_conv1(X_norm, edge_ix).relu()
        S = self.pool_conv2(S, edge_ix).relu()
        S = self.pool_conv3(S)

        _, _, Lc_geo, Lo_geo = dense_mincut_pool(X, A_geo, S)
        encoded, A_pool, Lc_net, Lo_net = dense_mincut_pool(X, A_net, S)

        X_rec = torch.matmul(S.softmax(dim=-1), encoded)

        X_rec = self.deconv3(X_rec).tanh()
        X_rec = self.deconv4(X_rec)
        decoded = X_rec.squeeze(0)

        return encoded, decoded, S.softmax(dim=-1), Lc_net, Lo_net, Lc_geo, Lo_geo, A_pool

In [7]:
# split train and validation data
X_E_train, X_E_val = train_test_split(X_E, test_size=0.2, random_state=42)

# normalize adjacency matrices
A_net = torch.tensor(nx.to_numpy_matrix(G_net), dtype=torch.float)
D_net = torch.tensor(np.diag(A_net.sum(axis=1)))
A_net = torch.tensor(inv(sqrtm(D_net)), dtype=torch.float) @ A_net @ torch.tensor(inv(sqrtm(D_net)), dtype=torch.float)

A_geo = torch.tensor(nx.to_numpy_matrix(G_geo, weight='distance'), dtype=torch.float)
D_geo = torch.tensor(np.diag(A_geo.sum(axis=1)))
A_geo = torch.tensor(inv(sqrtm(D_geo)), dtype=torch.float) @ A_geo @ torch.tensor(inv(sqrtm(D_geo)), dtype=torch.float)

## Training

In [8]:
clusters = 6 # choose number of spatial clusters

model = Spatial_AE(clusters)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction='mean')

Encoder Parameters: 3018
Decoder Parameters: 690
Total Parameters: 3708


In [9]:
epochs = 1000
train = list()
val = list()

In [10]:
with tqdm(total=epochs, desc='Progress', disable=False) as pbar:
    for epoch in range(epochs):
        optimizer.zero_grad()
        encoded, decoded, S, Lc_net, Lo_net, Lc_geo, Lo_geo,_ = model(X_E_train, edge_ix, A_net, A_geo)
        loss = criterion(decoded, X_E_train)
        train.append(loss.item())
        loss = loss + 0.5 * (Lc_net + Lo_net) + 0.5 * (Lc_geo + Lo_geo)
        loss.backward()
        optimizer.step()

        encoded, decoded, S, L_s, L_o, Ls_geo, Lo_geo,_ = model(X_E_val, edge_ix, A_net, A_geo)

        val_loss = criterion(decoded, X_E_val)
        val.append(val_loss.item())

        pbar.update(1)
        pbar.set_description("Validation Loss: " + str(val[-1]))

Validation Loss: 0.05319987237453461: 100%|████████████████████████████████████████| 1000/1000 [00:54<00:00, 18.34it/s]


## Saving Clusters

In [11]:
_, _, S, _, _, _, _, _ = model(X_E, edge_ix, A_net, A_geo)
C = S.mean(dim=0).argmax(dim=1).detach().numpy()
C = pd.DataFrame([list(range(len(G_net.nodes))), C]).astype(int).T
C.columns = ['Node', 'Cluster']
rows = list()
for i in range(len(G_net.nodes)):
    assignment = C.iloc[i]['Cluster']
    for node in G_net.nodes:
        rows.append([node, assignment])
C = pd.DataFrame(rows)
C.columns = ['Node', 'Cluster']

C.to_csv(f'spatial_cluster.csv', index=False)

# Temporal Aggregation Autoencoder

In [12]:
# augment power graph to create joint power/NG graph
NG_coords = pd.read_csv('Data/NG Network Topology/new_england_ng_nodes.csv')[['Lat','Lon']].to_numpy()
NG_edges = pd.read_csv('Data/Power Network Topology-full network (188 nodes)/NG_adj-Eelectricity nodes-188-nodes.csv')
NG_edges['NG Node'] += len(elec.columns)
for i in range(len(NG_edges['NG Node'])):
    G_net.add_node(NG_edges['NG Node'][i], pos=NG_coords[i])

for node in NG_edges['NG Node']:
    G_net.add_edge(node, node, distance=0)

for ix, row in NG_edges.iterrows():
    row = row.dropna().astype(int).tolist()
    for node in row[1:]:
        x,y = coords.loc[coords.bus_num == node][['lat','lon']].values.flatten()
        agg_node = coords.loc[(coords.lat == x) & (coords.lon == y)].bus_num.values.flatten()[0]
        G_net.add_edge(row[0], agg_node, distance=float(np.mean(distances)))

NG_edges = pd.read_csv('Data/NG Network Topology/new_england_ng_lines - unidirectional.csv')

NG_edges['from_node'] = NG_edges['from_node'].astype(int) + len(elec.columns)
NG_edges['to_node'] = NG_edges['to_node'].astype(int) + len(elec.columns)
G_net.add_edges_from(NG_edges[['from_node', 'to_node']].to_numpy())

NG_edge_index = torch.tensor(NG_edges[['from_node', 'to_node']].to_numpy(),dtype=torch.long).t().contiguous()
NG_edge_index -= len(elec.columns)

edge_ix = torch.tensor(np.array(nx.convert_node_labels_to_integers(G_net.copy()).edges),dtype=torch.long).t().contiguous()

In [13]:
class Temporal_AE(torch.nn.Module):

    def __init__(self):
        super(Temporal_AE, self).__init__()

        # Joint Convolutions
        self.encoder_joint_1 = ChebConv(73, 48, K=1)
        self.encoder_joint_2 = ChebConv(48, 3, K=1)

        encoder_params=0
        for p in list(self.parameters()):
            n=1
            for s in list(p.size()):
                n = n*s
            encoder_params += n
        print("Encoder Parameters: " + str(encoder_params))

        self.decoder_joint_1 = ChebConv(3, 73, K=1)

        # Electricity Block Decoder Parameters
        self.decoder_E1 = Linear(73, 73)
        self.decoder_E2 = Linear(73, 72)

        # Natural Gas Block Encoder Parameters
        self.decoder_NG1 = nn.Linear(73, 73)
        self.decoder_NG2 = nn.Linear(73, 1)

        decoder_params = 0
        for p in list(self.parameters()):
            n=1
            for s in list(p.size()):
                n = n*s
            decoder_params += n
        print("Decoder Parameters: " + str(decoder_params - encoder_params))

        print("Total Parameters: " + str(decoder_params))

    def forward(self, X_E, X_NG, edge_ix, return_cluster=False):

        X_NG = X_NG.reshape(-1,18,1)
        X = torch.cat([torch.block_diag(X_E[i], X_NG[i]).unsqueeze(0) for i in range(X_E.shape[0])])
        X = self.encoder_joint_1(X, edge_ix).tanh()
        X = self.encoder_joint_2(X, edge_ix)
        encoded = X.reshape(-1,106,3)

        X = self.decoder_joint_1(encoded, edge_ix)
        X_E, X_NG = torch.split(X, [88, 18], dim=1)

        X_E = X_E.reshape(-1,88,73)
        X_E = self.decoder_E1(X_E).tanh()
        X_E = self.decoder_E2(X_E)

        # natural gas decoder
        X_NG = X_NG.reshape(-1,18,73)
        X_NG = self.decoder_NG1(X_NG).tanh()
        X_NG = self.decoder_NG2(X_NG).reshape(-1,18)
       
        return encoded, X_E, X_NG

In [14]:
# concatenate electrical, wind, solar data
X = torch.cat([X_E, X_W, X_S], axis=2)

# split train and validation data
X_train, X_val, X_NG_train, X_NG_val = train_test_split(X, X_NG, test_size=0.2, random_state=42)

## Training

In [15]:
days = 7 # set number of representative days

model = Temporal_AE()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction='mean')

Encoder Parameters: 3699
Decoder Parameters: 16498
Total Parameters: 20197


In [16]:
alpha_G = 2
alpha_W = 0.5
alpha_S = 0.5

epochs = 1000
train = list()
val = list()

In [17]:
with tqdm(total=epochs, desc='Progress', disable=False) as pbar:
    for epoch in range(epochs):
        optimizer.zero_grad()
        X_encoded, X_decoded, X_NG_decoded = model(X_train, X_NG_train, edge_ix)
        loss = criterion(X_decoded[:,:,:24], X_train[:,:,:24]) + \
               alpha_G*criterion(X_NG_decoded, X_NG_train) + \
               alpha_W*criterion(X_decoded[:,:,24:48], X_train[:,:,24:48]) + \
               alpha_S*criterion(X_decoded[:,:,48:], X_train[:,:,48:])
        loss.backward()
        optimizer.step()
        train.append(loss.item())

        X_encoded, X_decoded, X_NG_decoded = model(X_val, X_NG_val, edge_ix)
        val_loss = criterion(X_decoded[:,:,:24], X_val[:,:,:24]) + \
               alpha_G*criterion(X_NG_decoded, X_NG_val) + \
               alpha_W*criterion(X_decoded[:,:,24:48], X_val[:,:,24:48]) + \
               alpha_S*criterion(X_decoded[:,:,48:], X_val[:,:,48:])
        val.append(val_loss.item())

        pbar.update(1)
        pbar.set_description("Validation Loss: " + str(val[-1]))

Validation Loss: 0.23755723237991333: 100%|████████████████████████████████████████| 1000/1000 [01:42<00:00,  9.78it/s]


## Saving Clusters

In [18]:
# retrieve latent embeddings from trained model
X_latent, _, _ = model(X, X_NG, edge_ix)
X_latent = X_latent.detach().numpy().reshape(365,-1)

# fit K-medoids
kmedoids = KMedoids(n_clusters=days, init='k-medoids++', max_iter=5000).fit(X_latent)
cluster_dates = np.unique(elec.index.date)[kmedoids.medoid_indices_]
row_list = list()

# create csv for learned cluster medians
for i in range(days):
    row = pd.DataFrame([list(np.unique(elec.index.date)).index(cluster_dates[i]), 
                        cluster_dates[i], 
                        sum(kmedoids.labels_ == i)]).T
    row_list.append(row)

kmedoids_results = pd.concat(row_list)
kmedoids_results.columns = ['Day of Year', 'Date', 'Weight']
kmedoids_results = kmedoids_results.sort_values(by='Day of Year')
kmedoids_results.to_csv('temporal_cluster.csv', index=False)